In [1]:
%load_ext autoreload
%autoreload 2

import gradio

SyntaxError: invalid syntax (networking.py, line 200)

In [3]:
def test(x):
    return x.upper()

def test2(x):
    return x.lower()

In [4]:
iface = gradio.Interface(inputs="textbox", outputs="textbox", model=test, model_type='function')

In [5]:
iface.launch(share=True)

NOTE: Gradio is in beta stage, please report all bugs to: a12d@stanford.edu
Model available locally at: http://localhost:7860/interface.html


NoSuchProcess: psutil.NoSuchProcess process no longer exists (pid=2744)